# Cleanng CDDB

In [2]:
import logging
from typing import Dict

import pandas as pd
import pandera as pa

import clean_cddb
from clean_cddb.utils import display_failure_cases_summary, get_check_func_descriptions

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(process)d - %(levelname)s - %(message)s",
)
filepath = "../data/input/cddb.tsv"
source_df = pd.read_csv(filepath, sep="\t", dtype="str")

## Apply validation checks (pandera schema) and review failure cases

In [3]:
try:
    validated_df = clean_cddb.schema(source_df, lazy=True)
    logging.info("Validation success. No failure cases detected.")
except pa.errors.SchemaErrors as err:
    logging.info("Validation failure. Failure cases detected.")
    logging.debug(err)
    failure_cases_df = err.failure_cases

failure_cases_df = failure_cases_df.pipe(get_check_func_descriptions, clean_cddb.schema)

2023-07-29 18:31:18,785 - 17800 - INFO - Validation failure. Failure cases detected.


`failure_cases_df`
* The `failure_cases_df` shows the name of the column, the check, failure case (example), and row index position of the failure case in the original data frame. 
* The index can support bulk operations such as joining and querying the original dataframe for failure cases or rejecting rows in the set of failure case indices.

In [5]:
(
    failure_cases_df.head().loc[
        :, ["schema_context", "column", "check", "failure_case", "index"]
    ]
)

,schema_context,column,check,failure_case,index
0,Column,artist,not_nullable,NaN,9030
4653,Column,genre,not_nullable,NaN,8088
4664,Column,genre,not_nullable,NaN,8135
4663,Column,genre,not_nullable,NaN,8134
4662,Column,genre,not_nullable,NaN,8127


### Summary of failure cases

Here we see aggregated counts of the number of failure cases for each validation check.

In [6]:
failure_cases_summary = (
    failure_cases_df.groupby(["column", "check"], as_index=False)
    .size()
    .sort_values(by=["column", "check"])
    .rename(columns={"size": "counts"})
)

failure_cases_summary

,column,check,counts
0,artist,Check for *possibly* invalid symbols.,1040
1,artist,Check for invalid artist values.,697
2,artist,not_nullable,1
3,category,Check for invalid categories.,89
4,genre,Check for invalid genres.,1
5,genre,not_nullable,3388
6,id,Check that the length of 'id' is 6 characters.,477
7,title,Check for *possibly* invalid symbols.,1200
8,title,not_nullable,8
9,year,Check if year is numeric.,28


We also have a helper utility function to display the source code along side each check function name.

In [5]:
# Report summary counts
display_failure_cases_summary(failure_cases_df)

+----------+------------------------------------------------+----------+--------------------------------------------------------------------------------------+
| column   | check                                          |   counts | check_source_code                                                                    |
+==========+================================================+==========+======================================================================================+
| artist   | Check for *possibly* invalid symbols.          |     1040 | def check_col_has_valid_characters(x: Any) -> bool:                                  |
|          |                                                |          |     """Check for *possibly* invalid symbols."""                                      |
|          |                                                |          |                                                                                      |
|          |                            

# Cleaning step

We can use the same checks from the pandera validation schema to trigger cleaning actions such as:
* do nothing / ignore the value
* transform the value; e.g., replace value with a substitute (e.g., 'N/A')
* or reject the entire record

Here we apply several cleaning functions on the source_df via .pipe(Callable).
* Each function takes a dataframe and returns a dataframe, so we can chain together the cleaning operations like so.
* Later, we will 
  1. compare `source_df` and `clean_df` as a before/after check
  2. re-apply our validation checks (pandera schema) to the new `clean_df` to verify that our transformations improved our data quality

In [7]:
def df_to_var(df, var_name):
    globals()[var_name] = df
    return df

clean_df = (
source_df
    .pipe(clean_cddb.clean_df_standardize_various_artists)
    .pipe(clean_cddb.clean_df_try_to_fix_encoding_errors, "artist")
    
    .pipe(df_to_var, 'clean_df_artist_transforms_only')
    
    .pipe(clean_cddb.clean_df_invalid_symbols)
    .pipe(clean_cddb.clean_df_invalid_categories)
    .pipe(clean_cddb.clean_df_id_zero_padding)
    .pipe(clean_cddb.clean_df_genre_invalid)
    .pipe(clean_cddb.clean_df_tracks_invalid_symbols)
    .pipe(clean_cddb.clean_df_year)
    .pipe(clean_cddb.clean_df_title)
    .pipe(clean_cddb.clean_df_genre)
    
    # Save an intermediate dataframe prior to dropping records
    # so we can compare with source_df later
    .pipe(df_to_var, 'clean_df_before_drops')
    
    # Drop rows with "REJECT_ROW*" prefix
    .query("~id.str.contains('REJECT_ROW')")
    
    .drop(columns=['merged_values'])
)

In [8]:
# apply schema to clean_df
try:
    validated_df = clean_cddb.schema(clean_df, lazy=True)
    logging.info("Validation success. No failure cases detected.")
except pa.errors.SchemaErrors as err:
    logging.info("Validation failure. Failure cases detected.")
    logging.debug(err)
    after_cleaning_failure_cases_df = err.failure_cases

after_cleaning_failure_cases_df = after_cleaning_failure_cases_df.pipe(
    get_check_func_descriptions, clean_cddb.schema
)

after_cleaning_failure_cases_summary = (
    after_cleaning_failure_cases_df.groupby(["column", "check"], as_index=False)
    .size()
    .sort_values(by=["column", "check"])
    .rename(columns={"size": "counts"})
)

2023-07-29 18:32:10,771 - 17800 - INFO - Validation failure. Failure cases detected.


# Evaluation

Counts of Failure Cases Before vs After Data Cleaning

In [9]:
(
failure_cases_summary
.merge(after_cleaning_failure_cases_summary, 
       on=['column', 'check'], 
       how='outer', 
       suffixes=['_before_cleaning', '_after_cleaning']
       )
.fillna('')
)

,column,check,counts_before_cleaning,counts_after_cleaning
0,artist,Check for *possibly* invalid symbols.,1040,
1,artist,Check for invalid artist values.,697,
2,artist,not_nullable,1,
3,category,Check for invalid categories.,89,
4,genre,Check for invalid genres.,1,
5,genre,not_nullable,3388,
6,id,Check that the length of 'id' is 6 characters.,477,
7,title,Check for *possibly* invalid symbols.,1200,641.0
8,title,not_nullable,8,
9,year,Check if year is numeric.,28,


Comparing `source_df` and `clean_df`

* We will actually use an intermediate dataframe `clean_df_before_drops` that has the same dimensions as our original dataframe.
    * Prior to dropping dirty records, `clean_df_before_drops` has values over-written with a prefix "REJECT_RECORD".
    * This enables easier side by side comparison.
* The final output `clean_df` will actually omit records that we intend to drop.


In [10]:
source_df.head()

,artist,category,genre,title,tracks,year,id,merged_values
0,Backstreet Boys,blues,Pop,Millennium,Larger Than Life | I Want It That Way | Show Me The Meaning Of Being Lonely | It's Gotta Be You | I Need You Tonight | Don't Want You Back | Don't Wanna Lose You Now | The One | Back To Your Heart | Spanish Eyes | No One Else Comes Close | The Perfect Fan | I'll Be There For You | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,NaN,10000,NaN
1,Various,data,NaN,Frankfurt Trance Vol. 04 cd1,DJ Tom Stevens VS. Fridge - Outface 2000 (Radio Mix) | Alice Deejay - Better Off Alone (Signum Remix) | Tillmann Uhrmacher Feat. Peter Ries - Bassfly (Original Mix) | DJ 2 L 8 - Too Late | Time Square - Invisible Girl (Future Breeze Remix) | Cirillo - Across The Soundline | DJ Leon & Jam X - Hold It | Sean Dexter - Synthetica (Extended Mix) | DJ BjÃ¶rn - On A Mission (Original Mix) | 8Voice - Music Hypnotizes 2000 | Alex Apollo - Jahr 2000 | Headroom - Utopia (Radio Mix) | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,NaN,100001,NaN
2,NO RETURN,data,Data,Self Mutilation,Do or Die | Truth and Reality | Lost | Soul Extractor | Sadistic Desire | The True Way | Fanatic Mind | Individualistic Ideal | One Life | Trail of Blood | Sect | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,NaN,100002,NaN
3,Ã¤Â¸Â­Ã¦?â€˜Ã©â€ºâ€¦Ã¤Â¿Å,data,Pop,Ã¦Æ’Â³Ã£?â€žÃ¥â€¡ÂºÃ£?Â®Ã£?â€¹Ã£?â€˜Ã£â€šâ€°,Ã§â€ºâ€ Ã¥Â¸Â°Ã£â€šÅ | Ã£?â€žÃ£?Â¤Ã£?â€¹Ã¨Â¡â€”Ã£?Â§Ã¤Â¼Å¡Ã£?Â£Ã£?Å¸Ã£?ÂªÃ£â€šâ€° | Ã©Â¢Â¨Ã£?Â®Ã£?ÂªÃ£?â€žÃ¦â€”Â¥ | Ã£?Å¸Ã£?Â Ã£?Å Ã¥â€°?Ã£?Å’Ã£?â€žÃ£?â€ž | Ã£?ÂµÃ£â€šÅ’Ã£?â€šÃ£?â€ž | Ã£?â€šÃ£â€š?Ã©?â€™Ã¦ËœÂ¥ | Ã¤Â¿ÂºÃ£?Å¸Ã£?Â¡Ã£?Â®Ã¦â€”â€¦ | Ã§â„¢Â½Ã£?â€žÃ¥Â¯Â«Ã§Å“Å¾Ã©Â¤Â¨ | Ã£?â€¢Ã£?â„¢Ã£â€šâ€°Ã£?â€žÃ¦â„¢â€šÃ¤Â»Â£ | Ã¥Â¤Å“Ã¨Â¡Å’Ã¥Ë†â€”Ã¨Â»Å | Ã£?â€šÃ£?ÂªÃ£?Å¸Ã£â€šâ€™Ã¦â€žâ€ºÃ£?â„¢Ã£â€šâ€¹Ã§Â§? | Ã©?â€™Ã¦ËœÂ¥Ã¨Â²Â´Ã¦â€”? | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,1989,100003,NaN
4,Emanuel,data,Data,Felicidade,Felicidade quando o telefone toca | Vem bailar o tic tic | Quero que sejas minha e de mais ninguem | Eu sei que me amas | O melhor que hÃ¡ | Minha vizinha deixa me a bater mal | S. JoÃ£o Ã© foliÃ£o | SÃ³ quero o teu carinho | tudo farei para ter a tua paixÃ£o | serÃ¡s sempre minha | Vem bailar o tic tic verÃ§Ã£o dance | Mix | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,1998,100004,NaN


In [11]:
clean_df_before_drops.head()

,artist,category,genre,title,tracks,year,id,merged_values
0,Backstreet Boys,blues,Pop,Millennium,Larger Than Life | I Want It That Way | Show Me The Meaning Of Being Lonely | It's Gotta Be You | I Need You Tonight | Don't Want You Back | Don't Wanna Lose You Now | The One | Back To Your Heart | Spanish Eyes | No One Else Comes Close | The Perfect Fan | I'll Be There For You | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,<NA>,010000,NaN
1,Various,N/A,N/A,Frankfurt Trance Vol. 04 cd1,DJ Tom Stevens VS. Fridge - Outface 2000 (Radio Mix) | Alice Deejay - Better Off Alone (Signum Remix) | Tillmann Uhrmacher Feat. Peter Ries - Bassfly (Original Mix) | DJ 2 L 8 - Too Late | Time Square - Invisible Girl (Future Breeze Remix) | Cirillo - Across The Soundline | DJ Leon & Jam X - Hold It | Sean Dexter - Synthetica (Extended Mix) | DJ BjÃ¶rn - On A Mission (Original Mix) | 8Voice - Music Hypnotizes 2000 | Alex Apollo - Jahr 2000 | Headroom - Utopia (Radio Mix) | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,<NA>,100001,NaN
2,NO RETURN,N/A,N/A,Self Mutilation,Do or Die | Truth and Reality | Lost | Soul Extractor | Sadistic Desire | The True Way | Fanatic Mind | Individualistic Ideal | One Life | Trail of Blood | Sect | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,<NA>,100002,NaN
3,REJECT_ROW - invalid artist,N/A,REJECT_ROW - invalid artist,REJECT_ROW - invalid artist,REJECT_ROW - invalid artist,<NA>,REJECT_ROW - invalid artist,REJECT_ROW - invalid artist
4,Emanuel,N/A,N/A,Felicidade,Felicidade quando o telefone toca | Vem bailar o tic tic | Quero que sejas minha e de mais ninguem | Eu sei que me amas | O melhor que hÃ¡ | Minha vizinha deixa me a bater mal | S. JoÃ£o Ã© foliÃ£o | SÃ³ quero o teu carinho | tudo farei para ter a tua paixÃ£o | serÃ¡s sempre minha | Vem bailar o tic tic verÃ§Ã£o dance | Mix | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |,1998,100004,NaN


In [12]:
pd.set_option('display.max_colwidth', 50)

comps_df = (
source_df
 .compare(clean_df_before_drops, result_names=('before_cleaning', 'after_cleaning'))
 .astype('object')
 .fillna('')
 )

In [13]:
comps_df.sample(10, random_state=3)

artist                                     category  \
        before_cleaning               after_cleaning before_cleaning   
8727                                                                   
3504                                                                   
3937                                                                   
20    Dingdong Avanzado  REJECT_ROW - invalid artist            data   
5267                                                                   
5929                                                                   
3681    Various Artists                      Various                   
7629    Various Artists                      Various                   
9702                                                                   
7414                                                                   

                              genre                               \
     after_cleaning before_cleaning               after_cleaning   
8727                                                         N/A   
3504                                                               
3937                                                               
20              N/A                  REJECT_ROW - invalid artist   
5267                                                               
5929                                                         N/A   
3681                                                               
7629                                                               
9702                                                               
7414                                                               

                              title                               \
                    before_cleaning               after_cleaning   
8727                                                               
3504                                                               
3937                                                               
20    The Best of Dingdong Avanzado  REJECT_ROW - invalid artist   
5267                                                               
5929                                                               
3681                                                               
7629                                                               
9702                                                               
7414                                                               

                                                 tracks  \
                                        before_cleaning   
8727                                                      
3504                                                      
3937                                                      
20    Ikaw Lamang | Tatlong Beinte Singko | Maghihin...   
5267                                                      
5929                                                      
3681                                                      
7629                                                      
9702                                                      
7414                                                      

                                             year                 \
                   after_cleaning before_cleaning after_cleaning   
8727                                                               
3504                                         1989           1989   
3937                                         2001           2001   
20    REJECT_ROW - invalid artist                                  
5267                                         1997           1997   
5929                                         1996           1996   
3681                                                               
7629                                                               
9702                                         2000           2000   
7414                                         1986           1

In [14]:
comps_df.sample(5, random_state=4)

artist                                         \
          before_cleaning                         after_cleaning   
16                                                                 
5512   Ã¥Â¼ÂµÃ¥Â­Â¸Ã¥?â€¹            REJECT_ROW - invalid artist   
8818                                                               
575                                                                
8510  Herbert von Karajan  REJECT_ROW - invalid symbol in tracks   

            category                                                  genre  \
     before_cleaning                         after_cleaning before_cleaning   
16              data                                    N/A                   
5512            rock                                    N/A      Soundtrack   
8818                                                                          
575                                                                           
8510       classical  REJECT_ROW - invalid symbol in tracks       Classical   

                                             \
                             after_cleaning   
16                                            
5512            REJECT_ROW - invalid artist   
8818                                    N/A   
575                                     N/A   
8510  REJECT_ROW - invalid symbol in tracks   

                                                  title  \
                                        before_cleaning   
16                                                        
5512  1999Ã¥?â€¹Ã¥â‚¬â€¹Ã¤ÂºÂºÃ¦Â¼â€Ã¥â€Â±Ã¦Å“Æ’ [...   
8818                                                      
575                                                       
8510                            95th Anniversary - CD 1   

                                             \
                             after_cleaning   
16                                            
5512            REJECT_ROW - invalid artist   
8818                                          
575                                           
8510  REJECT_ROW - invalid symbol in tracks   

                                                 tracks  \
                                        before_cleaning   
16                                                        
5512  Ã©â€¡â€¹Ã¦â€Â¾Ã¨â€¡ÂªÃ¥Â·Â± | Ã¤Â»Å Ã¦â„¢Â©Ã¨...   
8818                                                      
575                                                       
8510  Johann StrauÃŸ 2 - Der Zigeunerbaron, OuvertÃ¼...   

                                                       year                 \
                             after_cleaning before_cleaning after_cleaning   
16                                                     1996           1996   
5512            REJECT_ROW - invalid artist                                  
8818                                                                         
575                                                                          
8510  REJECT_ROW - invalid symbol in tracks                                  

                  id                                          merged_values  \
     before_cleaning                         after_cleaning before_cleaning   
16                                                                            
5512          105750            REJECT_ROW - invalid artist                   
8818                                                                          
575                                                                           
8510          108940  REJECT_ROW - invalid symbol in tracks                   

                                             
                             after_cleaning  
16                                           
5512            REJECT_ROW - invalid artist  
8818                                         
575                                          
8510  REJECT_ROW - invalid symbol in tracks

Smaller before/after example
* Here we can see that we transform "Various Artists" and "<various>" to "Various". 
* We also fixed invalid characters converting text from "JÃ¶rg Hilbert & Felix Janosa" to "Jörg Hilbert & Felix Janosa".
* Later, we will do a more comprehensive before/after analysis after applying all the cleaning transformations

In [15]:
example_idxs = [7629, 1822, 117, 4129]
source_df.compare(clean_df_artist_transforms_only).loc[example_idxs, :].fillna(pd.NA)

artist                             
                              self                        other
7629               Various Artists                      Various
1822  JÃ¶rg Hilbert & Felix Janosa  Jörg Hilbert & Felix Janosa
117                Various Artists                      Various
4129                     <various>                      Various

# End